普通方法

In [2]:
import torch

x = torch.tensor([1,2,3]).float()
A=torch.nn.Linear(3, 5)._parameters['weight']
B=torch.nn.Linear(5, 5)._parameters['weight']
C=torch.nn.Linear(5, 1)._parameters['weight']

x.matmul(A.T).matmul(B.T).matmul(C.T)

tensor([-0.0106], grad_fn=<SqueezeBackward4>)

权重运算符重载

In [152]:
class NetMat():
    def __init__(self,Weight=None,Moudle=None):
        if Weight !=None:
            self.weight=Weight
        elif Moudle !=None:
            self.weight=Moudle._parameters['weight'].T
        else:
            raise Exception('No Weight or Moudle')
        
    def __mul__(self, other):
        res = self.weight.matmul(other.weight)
        return NetMat(Weight=res)
    
    def __repr__(self) -> str:
        return f'NetMat(Weight={self.weight})'

x = torch.tensor([1,2,3]).float()
A=NetMat(Moudle=torch.nn.Linear(3, 5))
B=NetMat(Moudle=torch.nn.Linear(5, 5))
C=NetMat(Moudle=torch.nn.Linear(5, 1))

In [153]:
A*B*C

NetMat(Weight=tensor([[-0.0668],
        [ 0.0700],
        [ 0.0368]], grad_fn=<MmBackward0>))

In [154]:
x.matmul((A*B*C).weight)

tensor([0.1836], grad_fn=<SqueezeBackward4>)

网络运算符重载

In [2]:
from monet import *
F=Layer()
print(F)
F(torch.randn(2,5)),F

Layer(
  (Net): Linear(in_features=10, out_features=1, bias=True)
)


(tensor([[0.5537],
         [0.1506]], grad_fn=<AddmmBackward0>),
 Layer(
   (Net): Linear(in_features=5, out_features=1, bias=True)
 ))

In [1]:
from monet import *
F=SeqLayer()*Layer(0,1)
print(F)
F(torch.randn(2,5)),F

MoNet(
  (Net): Sequential(
    (0): SeqLayer(
      (Net): Sequential(
        (fc_1_0): Linear(in_features=10, out_features=64, bias=True)
        (fc_1_1): Linear(in_features=64, out_features=64, bias=True)
      )
    )
    (1): Layer(
      (Net): Linear(in_features=0, out_features=1, bias=True)
    )
  )
)


d:\Code\Skorch\.venv\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(tensor([[-0.2604],
         [ 0.0899]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): SeqLayer(
       (Net): Sequential(
         (fc_1_0): Linear(in_features=5, out_features=64, bias=True)
         (fc_1_1): Linear(in_features=64, out_features=64, bias=True)
       )
     )
     (1): Layer(
       (Net): Linear(in_features=64, out_features=1, bias=True)
     )
   )
 ))

In [1]:
from monet import *
F=Cell()*Layer(0,1)
print(F)
F(torch.randn(2,5)),F

MoNet(
  (Net): Sequential(
    (0): Cell(
      (Net): Sequential(
        (0:fc): Linear(in_features=10, out_features=1, bias=True)
        (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2:act): PReLU(num_parameters=1)
        (3:dp): Dropout(p=0.5, inplace=False)
      )
    )
    (1): Layer(
      (Net): Linear(in_features=0, out_features=1, bias=True)
    )
  )
)


d:\Code\Skorch\.venv\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(tensor([[-0.6335],
         [-0.6335]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:fc): Linear(in_features=5, out_features=1, bias=True)
         (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (2:act): PReLU(num_parameters=1)
         (3:dp): Dropout(p=0.5, inplace=False)
       )
     )
     (1): Layer(
       (Net): Linear(in_features=1, out_features=1, bias=True)
     )
   )
 ))

In [2]:
from monet import *
F=SeqCell()*Layer(0,1)
print(F)
F(torch.randn(2,5)),F

MoNet(
  (Net): Sequential(
    (0): SeqCell(
      (Net): Sequential(
        (cell-0): Sequential(
          (0:fc): Linear(in_features=10, out_features=10, bias=True)
          (1:bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:act): PReLU(num_parameters=1)
          (3:dp): Dropout(p=0.5, inplace=False)
        )
        (cell-1): Sequential(
          (0:fc): Linear(in_features=10, out_features=1, bias=True)
          (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:act): PReLU(num_parameters=1)
          (3:dp): Dropout(p=0.5, inplace=False)
        )
      )
    )
    (1): Layer(
      (Net): Linear(in_features=0, out_features=1, bias=True)
    )
  )
)


(tensor([[-0.1811],
         [-0.1811]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): SeqCell(
       (Net): Sequential(
         (cell-0): Sequential(
           (0:fc): Linear(in_features=5, out_features=10, bias=True)
           (1:bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2:act): PReLU(num_parameters=1)
           (3:dp): Dropout(p=0.5, inplace=False)
         )
         (cell-1): Sequential(
           (0:fc): Linear(in_features=10, out_features=1, bias=True)
           (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2:act): PReLU(num_parameters=1)
           (3:dp): Dropout(p=0.5, inplace=False)
         )
       )
     )
     (1): Layer(
       (Net): Linear(in_features=1, out_features=1, bias=True)
     )
   )
 ))

In [1]:
from monet import *
F=Mix()*Layer(1,1)
print(F)
F(torch.randn(2,5)),F

MoNet(
  (Net): Sequential(
    (0): Mix(
      (Net): Sequential(
        (0:input): Layer(
          (Net): Dropout(p=0.2, inplace=False)
        )
        (1:hiddens): SeqCell(
          (Net): Sequential(
            (cell-0): Sequential(
              (0:fc): Linear(in_features=10, out_features=32, bias=True)
              (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2:act): PReLU(num_parameters=1)
              (3:dp): Dropout(p=0.5, inplace=False)
            )
            (cell-1): Sequential(
              (0:fc): Linear(in_features=32, out_features=32, bias=True)
              (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2:act): PReLU(num_parameters=1)
              (3:dp): Dropout(p=0.5, inplace=False)
            )
          )
        )
        (2:output): Layer(
          (Net): Linear(in_features=32, out_features=1, bias=True)
        )
      )
    )
    (1)

(tensor([[-0.7559],
         [-0.7585]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Mix(
       (Net): Sequential(
         (0:input): Layer(
           (Net): Dropout(p=0.2, inplace=False)
         )
         (1:hiddens): SeqCell(
           (Net): Sequential(
             (cell-0): Sequential(
               (0:fc): Linear(in_features=5, out_features=32, bias=True)
               (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
               (2:act): PReLU(num_parameters=1)
               (3:dp): Dropout(p=0.5, inplace=False)
             )
             (cell-1): Sequential(
               (0:fc): Linear(in_features=32, out_features=32, bias=True)
               (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
               (2:act): PReLU(num_parameters=1)
               (3:dp): Dropout(p=0.5, inplace=False)
             )
           )
         )
         (2:output): Layer(
       

In [1]:
from monet import *
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_img=Mix(0,[[16,32],1],[['cv2','bn2','mp2','act'],'fl']).Net
        self.txt_fc=Mix(0,[[32,64]],[['fc','bn','act','dp']]).Net
        self.out_mlp=Mix(0,[1],['fc']).Net

    def forward(self, img, x):
        y1=self.cnn_img(img)
        y2=self.txt_fc(x)
        y3=torch.cat([y1,y2],dim=1)
        y=self.out_mlp(y3)
        return y
F=Net()
print(F)
F(torch.randn(2,1,64,64),torch.randn(2,10)),F

Net(
  (cnn_img): Sequential(
    (0:input): SeqCell(
      (Net): Sequential(
        (cell-0): Sequential(
          (0:cv): Conv2d(0, 16, kernel_size=(3, 3), stride=(1, 1))
          (1:bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (3:act): PReLU(num_parameters=1)
        )
        (cell-1): Sequential(
          (0:cv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
          (1:bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (3:act): PReLU(num_parameters=1)
        )
      )
    )
    (1:output): Layer(
      (Net): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (txt_fc): Sequential(
    (0:input): SeqCell(
      (Net): Sequential(
        (cell-0): Sequential(
          (0:fc): Linear(in_features=0, ou

d:\Code\Skorch\.venv\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(tensor([[1.4184],
         [0.5706]], grad_fn=<AddmmBackward0>),
 Net(
   (cnn_img): Sequential(
     (0:input): SeqCell(
       (Net): Sequential(
         (cell-0): Sequential(
           (0:cv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
           (1:bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
           (3:act): PReLU(num_parameters=1)
         )
         (cell-1): Sequential(
           (0:cv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
           (1:bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
           (3:act): PReLU(num_parameters=1)
         )
       )
     )
     (1:output): Layer(
       (Net): Flatten(start_dim=1, end_dim=-1)
     )
   )
   (txt_fc): Sequential(
     (0:input): SeqCell(
       

In [2]:
from monet import *
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = Layer(10, 20, 'fc')
        self.fc2 = Layer(20, 1, 'fc')

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

Net()

Net(
  (fc1): Layer(
    (Net): Linear(in_features=10, out_features=20, bias=True)
  )
  (fc2): Layer(
    (Net): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [4]:
A = Cell()
print(A)
q=A(torch.tensor([[0.0,0.0],[1.0,1.0]]))
print(q)
print(A)

Cell(
  (Net): Sequential(
    (0:fc): Linear(in_features=10, out_features=1, bias=True)
    (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2:act): PReLU(num_parameters=1)
    (3:dp): Dropout(p=0.5, inplace=False)
  )
)
tensor([[0.],
        [-0.]], grad_fn=<MulBackward0>)
Cell(
  (Net): Sequential(
    (0:fc): Linear(in_features=2, out_features=1, bias=True)
    (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2:act): PReLU(num_parameters=1)
    (3:dp): Dropout(p=0.5, inplace=False)
  )
)


In [5]:
import torch
from MoNet.monet import Cell, Layer

F=Layer(1,10)*Cell(10,10)*Layer(10,1)
print(F)
F(torch.tensor([[0.0],
                [1.0]]))

MoNet(
  (Net): Sequential(
    (0): Layer(
      (Net): Linear(in_features=1, out_features=10, bias=True)
    )
    (1): Cell(
      (Net): Sequential(
        (0:fc): Linear(in_features=10, out_features=10, bias=True)
        (1:bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2:act): PReLU(num_parameters=1)
        (3:dp): Dropout(p=0.5, inplace=False)
      )
    )
    (2): Layer(
      (Net): Linear(in_features=10, out_features=1, bias=True)
    )
  )
)


tensor([[0.2051],
        [0.9309]], grad_fn=<AddmmBackward0>)

In [6]:
from monet import *

Fc=Cell(1,5,['fc','act'])
Hid1=Cell(5,1,['fc','act'])
Hid2=Cell(5,1,['fc','act'])
Out = Layer(2,1,'fc')

x=torch.tensor([[0.0]])
F1=(Fc*Hid1+Fc*Hid2)*Out
F2=Fc*(Hid1+Hid2)*Out
F1(x)==F2(x)

tensor([[True]])

In [7]:
from monet import *
cell()*cell()

MoNet(
  (Net): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=1)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): PReLU(num_parameters=1)
    (7): Dropout(p=0.5, inplace=False)
  )
)

In [11]:
Mix()

Mix(
  (Net): Sequential(
    (0:input): Layer(
      (Net): Dropout(p=0.2, inplace=False)
    )
    (1:hiddens): SeqCell(
      (Net): Sequential(
        (cell-0): Sequential(
          (0:fc): Linear(in_features=10, out_features=32, bias=True)
          (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:act): PReLU(num_parameters=1)
          (3:dp): Dropout(p=0.5, inplace=False)
        )
        (cell-1): Sequential(
          (0:fc): Linear(in_features=32, out_features=32, bias=True)
          (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:act): PReLU(num_parameters=1)
          (3:dp): Dropout(p=0.5, inplace=False)
        )
      )
    )
    (2:output): Layer(
      (Net): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)

In [12]:
from monet import *
Mix(2,[10,10,10,10],['fc','bn','act','fl'])

Mix(
  (Net): Sequential(
    (0:input): Layer(
      (Net): Linear(in_features=2, out_features=10, bias=True)
    )
    (1:hiddens): Layer(
      (Net): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2:hiddens): Layer(
      (Net): PReLU(num_parameters=1)
    )
    (3:output): Layer(
      (Net): Flatten(start_dim=1, end_dim=-1)
    )
  )
)

In [14]:
Mix(0,[[10,20],0],[['cv2','bn2','act'],'fl']).set_i(1,2,8,8)

Mix(
  (Net): Sequential(
    (0:input): SeqCell(
      (Net): Sequential(
        (cell-0): Sequential(
          (0:cv): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1))
          (1:bn): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:act): PReLU(num_parameters=1)
        )
        (cell-1): Sequential(
          (0:cv): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
          (1:bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2:act): PReLU(num_parameters=1)
        )
      )
    )
    (1:output): Layer(
      (Net): Flatten(start_dim=1, end_dim=-1)
    )
  )
)

In [1]:
from monet import *
B=((Mix(0,[[10,20],0],[['cv2','bn2','act'],'fl'])*
   Mix(0,[10],[['fc','act','fl']]))*
   Mix(0,[10],[['fc','act']])).set_i((1,2,8,8))
B

d:\Code\Skorch\.venv\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


MoNet(
  (Net): Sequential(
    (0): Mix(
      (Net): Sequential(
        (0:input): SeqCell(
          (Net): Sequential(
            (cell-0): Sequential(
              (0:cv): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1))
              (1:bn): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2:act): PReLU(num_parameters=1)
            )
            (cell-1): Sequential(
              (0:cv): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
              (1:bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2:act): PReLU(num_parameters=1)
            )
          )
        )
        (1:output): Layer(
          (Net): Flatten(start_dim=1, end_dim=-1)
        )
      )
    )
    (1): Mix(
      (Net): Sequential(
        (0:input): Cell(
          (Net): Sequential(
            (0:fc): Linear(in_features=320, out_features=10, bias=True)
            (1:act): PReLU(num_parameters=1)
      

In [2]:
from monet import *
A=Layer(net='cv').set_i(1,2,3,3)
A

Layer(
  (Net): Conv2d(2, 1, kernel_size=(3, 3), stride=(1, 1))
)

In [1]:
from monet import *
A=MoNet(nn.Linear(10,10))
A.set_i(2)

MoNet(
  (Net): Linear(in_features=2, out_features=10, bias=True)
)

In [10]:
A=layer()
A2=layer()
id(A),id(A2)

(3077652475968, 3077652479280)

In [11]:
id((A*1)),id((A*2).Net[0]),id((A*2).Net[1]) # 浅拷贝

(3077652475968, 3077652475968, 3077652475968)

In [12]:
B=A*A*A2  # 浅拷贝
id(B.Net[0]),id(B.Net[1]),id(B.Net[2])

(3077652475968, 3077652475968, 3077652479280)

In [13]:
id((A**1)),id((A**2).Net[0]),id((A**2).Net[1]) # 深拷贝

(3077652472080, 3077652473616, 3077652472080)

In [14]:
B=(A**A)**3  # 深拷贝
id(B.Net[0].Net[0]),id(B.Net[1].Net[0]),id(B.Net[2].Net[0])

(3077652744368, 3077652745136, 3077652746000)

In [15]:
id((A+A)),id((A+A).Net[0]),id((A+A).Net[1]) # 浅拷贝

(3079367948560, 3077652475968, 3077652475968)

In [17]:
id((A&A)),id((A&A).Net[0]),id((A&A).Net[1]) # 深拷贝

(3079366077712, 3077652744320, 3077652747920)

自适应调整输入形状
- 设置输入维度为0，用set_i()或在首次forward时自适应调整顶层输入形状

In [1]:
from monet import *
F=Cell(0,10,['fc','act','fc','act','fc','act'])*layer(0,1)
F(torch.tensor([[0.0]])),F

d:\Code\Skorch\.venv\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(tensor([[-0.5330]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:fc): Linear(in_features=1, out_features=10, bias=True)
         (1:act): PReLU(num_parameters=1)
         (2:fc): Linear(in_features=10, out_features=10, bias=True)
         (3:act): PReLU(num_parameters=1)
         (4:fc): Linear(in_features=10, out_features=10, bias=True)
         (5:act): PReLU(num_parameters=1)
       )
     )
     (1): MoNet(
       (Net): Linear(in_features=10, out_features=1, bias=True)
     )
   )
 ))

In [2]:
F.set_i(2)(torch.tensor([[0.0,1.0]])),F

(tensor([[-0.4637]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:fc): Linear(in_features=2, out_features=10, bias=True)
         (1:act): PReLU(num_parameters=1)
         (2:fc): Linear(in_features=10, out_features=10, bias=True)
         (3:act): PReLU(num_parameters=1)
         (4:fc): Linear(in_features=10, out_features=10, bias=True)
         (5:act): PReLU(num_parameters=1)
       )
     )
     (1): MoNet(
       (Net): Linear(in_features=10, out_features=1, bias=True)
     )
   )
 ))

In [1]:
from monet import *
F=Cell(2,10,['cv','fl'])*layer(0,1)
F(torch.randn(1,2,3,3)),F

d:\Code\Skorch\.venv\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(tensor([[0.4370]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:cv): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1))
         (1:fl): Flatten(start_dim=1, end_dim=-1)
       )
     )
     (1): MoNet(
       (Net): Linear(in_features=10, out_features=1, bias=True)
     )
   )
 ))

In [2]:
# set_i可强制改变输入层维度
F.set_i(1,3,6,6)(torch.randn(1,3,6,6)),F

(tensor([[0.0472]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:cv): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
         (1:fl): Flatten(start_dim=1, end_dim=-1)
       )
     )
     (1): MoNet(
       (Net): Linear(in_features=160, out_features=1, bias=True)
     )
   )
 ))